In [3]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import OllamaEmbeddings
from langchain.chains import LLMChain
from langchain_community.chat_models import ChatOllama
from langchain.prompts import PromptTemplate
import requests
import os

In [4]:
# Example: "The Tale of Peter Rabbit" by Beatrix Potter (Project Gutenberg)
url = "https://www.gutenberg.org/cache/epub/14838/pg14838.txt"
response = requests.get(url)
with open("book.txt", "w", encoding="utf-8") as f:
    f.write(response.text)
print("Book downloaded and saved as book.txt")

Book downloaded and saved as book.txt


In [5]:
loader = TextLoader("book.txt")
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = text_splitter.split_documents(documents)
print(f"Total chunks: {len(docs)}")

Total chunks: 69


In [6]:
embedding_model = OllamaEmbeddings(model="llama3")
vectorstore = FAISS.from_documents(docs, embedding_model)
retriever = vectorstore.as_retriever()

/var/folders/m_/5kg3cz2s7lq7__mtwkrfb5pc0000gn/T/ipykernel_33082/3969312578.py:1: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embedding_model = OllamaEmbeddings(model="llama3")


In [10]:
quiz_prompt = PromptTemplate(
    input_variables=["context", "difficulty"],
    template="""
You are a creative quiz maker for children aged 4-8.
Based on the following book excerpt, generate ONE multiple-choice question that is playful, uses emojis, and has 4 answer options.
The question should be {difficulty} difficulty.
Mark the correct answer with <-- correct.

Excerpt:
{context}

Format:
Q: <question>
A) <option1>
B) <option2>
C) <option3>
D) <option4>
"""
)

In [11]:
llm = ChatOllama(model="llama3")
chain = LLMChain(llm=llm, prompt=quiz_prompt)

difficulty_levels = ["Easy", "Medium", "Hard"]
questions_by_difficulty = {d: [] for d in difficulty_levels}

for difficulty in difficulty_levels:
    print(f"=== Generating {difficulty} questions ===")
    for i, doc in enumerate(docs[:5]):  # Adjust number as needed
        print(f"Generating {difficulty} question from chunk {i+1}...")
        result = chain.run(context=doc.page_content, difficulty=difficulty)
        questions_by_difficulty[difficulty].append(result)
        print(result)
        print("="*40)

=== Generating Easy questions ===
Generating Easy question from chunk 1...
Here's a playful multiple-choice question based on the excerpt:

Q: What is special about this book? 📚👀
A) It has no pictures! 😔
B) Anyone can read it for free! 🎉 <-- correct
C) You need to buy it to own it! 💸
D) It's only for kids in the United States! 🇺🇸
Generating Easy question from chunk 2...
Here's a multiple-choice question based on the given excerpt:

Q: 📖 What do you need to do before reading an eBook? 🤔
A) Just start reading it anywhere!
B) <-- correct Use headphones and a flashlight! 
C) Ask your friend if they want to read it with you!
D) Make some popcorn and grab a snack!
Generating Easy question from chunk 3...
🐰🌱 Here's a question for you! 🤔

What did Peter Rabbit love to do? 🎉

A) Go swimming in the river <-- correct
B) Play with his friends in the forest
C) Eat delicious carrots all day
D) Take long naps in the sun

🐰👀
Generating Easy question from chunk 4...
Here's a playful multiple-choice que

In [12]:
parsed_questions_by_difficulty = {}
for difficulty, questions in questions_by_difficulty.items():
    parsed = [parse_quiz_output(q) for q in questions]
    parsed = [q for q in parsed if q is not None]
    parsed_questions_by_difficulty[difficulty] = parsed

import json
print(json.dumps(parsed_questions_by_difficulty, indent=2, ensure_ascii=False))

{
  "Easy": [
    {
      "question": "What is special about this book? 📚👀",
      "options": [
        "It has no pictures! 😔",
        "Anyone can read it for free! 🎉",
        "You need to buy it to own it! 💸",
        "It's only for kids in the United States! 🇺🇸"
      ],
      "answer_idx": 1
    },
    {
      "question": "📖 What do you need to do before reading an eBook? 🤔",
      "options": [
        "Just start reading it anywhere!",
        "Use headphones and a flashlight!",
        "Ask your friend if they want to read it with you!",
        "Make some popcorn and grab a snack!"
      ],
      "answer_idx": 1
    },
    {
      "question": "What are the names of the four little Rabbits mentioned in the story? 🐰🏠",
      "options": [
        "Benny, Billy, Bob, and Barbara",
        "Flopsy, Mopsy, Cotton-tail, and Peter",
        "Rosie, Ruby, Rusty, and Rex",
        "Sammy, Sally, Sasha, and Squeaky"
      ],
      "answer_idx": 1
    },
    {
      "question": "What did 